# Create speech-enabled apps with Azure AI services

The Azure AI Speech service enables you to build speech-enabled applications. This module focuses on using the speech-to-text and text to speech APIs, which enable you to create apps that are capable of speech recognition and speech synthesis.

- Provision an Azure resource for the Azure AI Speech service
- Use the Azure AI Speech to text API to implement speech recognition
- Use the Text to speech API to implement speech synthesis
- Configure audio format and voices
- Use Speech Synthesis Markup Language (SSML)

# Importing libraries

In [5]:
from dotenv import load_dotenv
from datetime import datetime
import os

In [6]:
import azure.cognitiveservices.speech as speech_sdk

# Loading configurations

In [7]:
load_dotenv()
ai_key = os.getenv('SPEECH_KEY')
ai_region = os.getenv('SPEECH_REGION')

True

# Configure speech service

In [8]:
speech_config = speech_sdk.SpeechConfig(ai_key, ai_region)
print('Ready to use speech service in:', speech_config.region)

Ready to use speech service in: eastus
